In [1]:
import os
import json
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models


In [2]:

# JSON 파일 로드 함수
def load_json(json_path):
    with open(json_path, 'r') as json_file:
        return json.load(json_file)

# 데이터 로딩 및 전처리
data_path = "/Users/brianmg/Desktop/New_Sample/라벨링데이터/TL01/반려묘/피부/일반카메라/유증상/A2_비듬_각질_상피성잔고리"  # 데이터 폴더 경로 설정
image_paths = [file for file in os.listdir(data_path) if file.endswith('.jpg')]
label_paths = [file for file in os.listdir(data_path) if file.endswith('.json')]


In [ ]:

# 데이터셋 구성
X_data = []
Y_labels = []

for image_path in image_paths:
    # 이미지 로드 및 전처리
    image = cv2.imread(os.path.join(data_path, image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image / 255.0  # 정규화

    # JSON 파일로부터 라벨 로드
    json_filename = image_path.replace('.jpg', '.json')
    json_path = os.path.join(data_path, json_filename)
    json_data = load_json(json_path)

    # 라벨 정보 추출
    labels = []
    for label_info in json_data['labelingInfo']:
        if 'polygon' in label_info:
            label = label_info['polygon']['label']
            vertices = label_info['polygon']['location'][0]
            labels.append({'label': label, 'vertices': vertices})
        elif 'box' in label_info:
            label = label_info['box']['label']
            box_info = label_info['box']['location'][0]
            x, y, w, h = box_info['x'], box_info['y'], box_info['width'], box_info['height']
            vertices = np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]])
            labels.append({'label': label, 'vertices': vertices})

    # 데이터셋에 추가
    X_data.append(image)
    Y_labels.append(labels)

In [ ]:
# 데이터셋 분리 (필요에 따라 train/test를 적절히 조절)
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_labels, test_size=0.2, random_state=42)

# 모델 정의
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image.shape[0], image.shape[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='softmax')  # 클래스 수는 라벨의 개수에 따라 다름
])

In [ ]:
for image_path in image_paths:
    # 이미지 로드 및 전처리
    image = cv2.imread(os.path.join(data_path, image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image / 255.0  # 정규화

    # JSON 파일로부터 라벨 로드
    json_filename = image_path.replace('.jpg', '.json')
    json_path = os.path.join(data_path, json_filename)
    json_data = load_json(json_path)

    # 라벨 정보 추출
    labels = []
    for label_info in json_data['labelingInfo']:
        if 'polygon' in label_info:
            label = label_info['polygon']['label']
            vertices = label_info['polygon']['location'][0]
            labels.append({'label': label, 'vertices': vertices})
        elif 'box' in label_info:
            label = label_info['box']['label']
            box_info = label_info['box']['location'][0]
            x, y, w, h = box_info['x'], box_info['y'], box_info['width'], box_info['height']
            vertices = np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]])
            labels.append({'label': label, 'vertices': vertices})

    # 데이터셋에 추가
    X_data.append(image)
    Y_labels.append(labels)

In [ ]:

# 데이터셋 분리 (필요에 따라 train/test를 적절히 조절)
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_labels, test_size=0.2, random_state=42)

# 모델 정의
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image.shape[0], image.shape[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='softmax')  # 클래스 수는 라벨의 개수에 따라 다름
])

In [ ]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(np.array(X_train), np.array(Y_train), epochs=10, validation_data=(np.array(X_test), np.array(Y_test)))

# 모델 평가
test_loss, test_acc = model.evaluate(np.array(X_test), np.array(Y_test))
print(f"Test Accuracy: {test_acc}")